In [1]:
from rdkit import Chem
from config import MOPACMozymeConfig

mol = Chem.MolFromSmiles("C=CC=O")

config = MOPACMozymeConfig()
config.add_molecule(mol)

print(config.get_config_str())

PDBOUT MOZYME GEO-OK SETPI CHARGE=0
MOPACMozymeConfig 
| Unnamed 
HETATM    1  C1  UNL     1       0.000   0.000   0.000  1.00  0.00           C  
HETATM    2  C2  UNL     1       0.000   0.000   0.000  1.00  0.00           C  
HETATM    3  C3  UNL     1       0.000   0.000   0.000  1.00  0.00           C  
HETATM    4  O1  UNL     1       0.000   0.000   0.000  1.00  0.00           O  

1 2
3 4


In [2]:
from rdkit import Chem
from interface import MOPACInterface, MOPACMozymeConfig

mol = Chem.MolFromSmiles("C=CC=O")

interface = MOPACInterface(config=MOPACMozymeConfig())

interface.update_config(mol)

out_path, arc_path = interface.run_job(
    base_dir="./mopac_runs",
    debug=True
)

MOPAC input file written: mopac_runs/YLKJDX.mop


          MOPAC Job: "YLKJDX.mop" ended normally on Apr 22, 2025, at 08:25.



In [3]:
import os
from rdkit import Chem
from rdkit.Chem import AllChem
from config import MOPACConfig
from optimizer import MOPACOptimizer
from energy import MOPACEnergyCalculator

def test_mopac_optimization_and_energy():

    smi = "CCO"
    mol = Chem.MolFromSmiles(smi)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    
    opt_config = MOPACConfig()
    opt_config.keywords.extend(["PM7", "PRECISE"])
    optimizer = MOPACOptimizer(config=opt_config)

    opt_results = optimizer.run_optimization(mol, debug=True)
    print(f"{opt_results['pre_energy']:.2f}")
    print(f"{opt_results['post_energy']:.2f}")
    print(f"{opt_results['coordinates'].shape}")

    energy_config = MOPACConfig()
    energy_config.keywords.extend(["PM7", "1SCF"])
    energy_calc = MOPACEnergyCalculator(config=energy_config)
    
    energy_results = energy_calc.calculate_energy(mol, debug=True)
    print(f"{energy_results['energy']:.2f}")
    
test_mopac_optimization_and_energy()

MOPAC input file written: LZGMTN.mop


          MOPAC Job: "LZGMTN.mop" ended normally on Apr 22, 2025, at 08:27.

-54.56
-57.86
(9, 3)
MOPAC input file written: A20SAN.mop


          MOPAC Job: "A20SAN.mop" ended normally on Apr 22, 2025, at 08:27.

-54.56
